# 1. Load the data

In [1]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
import random

In [2]:
questions = pd.read_csv('./data/qs_2016.csv')

In [3]:
questions = questions.drop(['Unnamed: 0'], axis=1 )

In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('./data/intents.json').read()
intents = json.loads(data_file)

In [5]:
questions_sample = questions.sample(200)

In [6]:
pattern_question=[]
for index, question in questions_sample.iterrows():
        pattern_question.append(question.Title)
        a = {
        'tag': 'question',
        'patterns': pattern_question,
        'responses': [""],
        'context':[""]
        }
        intents['intents'].append(a)

# 2. Preprocess data

In [7]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [8]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

40037 documents
5 classes ['goodbye', 'greeting', 'options', 'question', 'thanks']
881 unique lemmatized words ['#', '$', '&', "'", "''", "'s", "'type", '(', ')', '*', '+', ',', '-', '.', '.ajax', '.htaccess', '.net', '/', '1d-array', '2', '2d', '3', '3044', '3d', '4.8.1', '407', '5', '5.2', '6', '8.5', ':', ':before', ':call', ':invokeargs', ':thread', '>', '[', ']', '_controller', '`', '``', 'a', 'accesible', 'access', 'accessibilty', 'accessing', 'according', 'accounted', 'acitivty', 'across', 'act', 'activity', 'actor', 'ad', 'adapter', 'add', 'addon/extension', 'advantage', 'aem', 'after', 'again', 'against', 'ahead', 'ajax', 'akka', 'algorithm', 'all', 'allows', 'am', 'amazon', 'an', 'analysis', 'anchor', 'and', 'android', 'angular', 'angularjs', 'another', 'any', 'anyone', 'apache', 'api', 'app', 'appdelegate', 'application', 'arbitrary', 'are', 'array', 'artisan', 'asp.net', 'assistance', 'at', 'atom', 'attribute', 'audio', 'authentication', 'auto', 'autocomplete', 'autoconnect

# 3. Create training and testing data

In [9]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-9-5117620404a0>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


# 4. Build the model

In [10]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [11]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

/home/guedri/anaconda3/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/100
8008/8008 [==============================] - 12s 1ms/step - loss: 0.0111 - accuracy: 0.9985
Epoch 2/100
8008/8008 [==============================] - 11s 1ms/step - loss: 0.0058 - accuracy: 0.9991
Epoch 3/100
8008/8008 [==============================] - 12s 2ms/step - loss: 0.0035 - accuracy: 0.9991
Epoch 4/100
8008/8008 [==============================] - 14s 2ms/step - loss: 0.0026 - accuracy: 0.9991
Epoch 5/100
8008/8008 [==============================] - 11s 1ms/step - loss: 0.0020 - accuracy: 0.9991
Epoch 6/100
8008/8008 [==============================] - 12s 1ms/step - loss: 0.0018 - accuracy: 0.9991
Epoch 7/100
8008/8008 [==============================] - 13s 2ms/step - loss: 0.0015 - accuracy: 0.9993
Epoch 8/100
8008/8008 [==============================] - 12s 2ms/step - loss: 0.0015 - accuracy: 0.9995
Epoch 9/100
8008/8008 [==============================] - 13s 2ms/step - loss: 0.0013 - accuracy: 0.9995
Epoch 10/100
8008/8008 [==============================] - 14s 2m

8008/8008 [==============================] - 11s 1ms/step - loss: 1.1939e-04 - accuracy: 1.0000
Epoch 78/100
8008/8008 [==============================] - 11s 1ms/step - loss: 2.3699e-04 - accuracy: 1.0000
Epoch 79/100
8008/8008 [==============================] - 11s 1ms/step - loss: 2.7774e-04 - accuracy: 0.9999
Epoch 80/100
8008/8008 [==============================] - 11s 1ms/step - loss: 3.5540e-04 - accuracy: 0.9999
Epoch 81/100
8008/8008 [==============================] - 11s 1ms/step - loss: 1.5248e-04 - accuracy: 1.0000
Epoch 82/100
8008/8008 [==============================] - 12s 1ms/step - loss: 2.2224e-04 - accuracy: 1.0000
Epoch 83/100
8008/8008 [==============================] - 12s 2ms/step - loss: 1.3948e-04 - accuracy: 1.0000
Epoch 84/100
8008/8008 [==============================] - 12s 2ms/step - loss: 2.9871e-04 - accuracy: 0.9999
Epoch 85/100
8008/8008 [==============================] - 12s 1ms/step - loss: 1.8545e-04 - accuracy: 1.0000
Epoch 86/100
8008/8008 [========

In [12]:
with open('intents.json', 'w') as fp:
    json.dump(intents, fp)